# Markov

**Algoritmo para criação de uma HMM**
1. Defina o espaço de estados e o espaço de observações
2. Defina a distribuição do estado inicial
3. Defina a probabilidade das transições de estado
4. Defina a probabilidade de uma observação dada um estado
5. Crie o modelo
6. Dada uma observação, retorne a sequência mais provável de estados
escondidos
7. Avalie o modelo

## Contexto

- Observações: Palavras do vocabulário das letras.
- Estados Ocultos: Classes Gramaticais (POS tags).
- π (distribuição inicial): Probabilidade de cada classe gramatical iniciar um verso/frase.
- A (matriz de transição): Probabilidade de uma classe gramatical seguir outra.
- B (matriz de emissão): Probabilidade de uma palavra específica ser gerada por uma determinada classe gramatical.

## Leitura dos Dados

A função `get_text_list_from_files` recebe o nome da pasta e retorna os conteúdos dos arquivos como uma lista de strings.

In [9]:
import os
import glob


def get_text_list_from_files( folder_name: str ) -> list[ str ]:
    """
    Lê o conteúdo de múltiplos arquivos e retorna uma lista de strings,
    onde cada string representa o conteúdo de um arquivo.

    Args:
        folder_name (str): O nome da pasta onde os arquivos de texto estão localizados.
                           Espera-se que os arquivos estejam dentro de um subdiretório chamado 'musicas'.

    Returns:
        list[str]: Uma lista onde cada elemento é o conteúdo dos arquivos lidos.
    """

    # Utiliza a biblioteca glob para encontrar todos os arquivos com extensão .txt
    # dentro do subdiretório especificado dentro de 'musicas'.
    files = glob.glob( f"musicas/{folder_name}/*.txt" )

    text_list: list[ str ] = [ ]
    for file_path in files:
        with open( file_path, "r", encoding = "utf-8" ) as file:
            # Lê todo o conteúdo do arquivo de uma vez.
            # Se o arquivo contiver várias linhas e você precisar de cada linha como um item separado na lista,
            # você pode iterar sobre o objeto 'file' (por exemplo, `for line in file: text_list.append(line.strip())`).
            text_list.append( file.read() )
    return text_list


# Cria o DataSet de treino chamando a função com o nome da pasta 'train'.
train_ds = get_text_list_from_files( "train" )

# Cria o DataFrame de teste/validação chamando a função com o nome da pasta 'test'.
test_ds = get_text_list_from_files( "test" )

# Imprime o tamanho (número de linhas) do DataFrame de treino.
print( f"Tamanho do DataFrame de Treino: {len( train_ds )}" )

# Imprime o tamanho (número de linhas) do DataFrame de teste/validação.
print( f"Tamanho do DataFrame de Teste/Validação: {len( test_ds )}" )

Tamanho do DataFrame de Treino: 1444
Tamanho do DataFrame de Teste/Validação: 361


## Pré-processamento dos Dados

In [10]:
import spacy

# Carregando o modelo português do spaCy
try:
    nlp = spacy.load( "pt_core_news_lg" )
except OSError:
    print( "Modelo 'pt_core_news_lg' não encontrado. Baixando..." )
    os.system( "python -m spacy download pt_core_news_lg" )
    nlp = spacy.load( "pt_core_news_lg" )


def tag_sequences( text_list: list[ str ] ) -> list[ list[ tuple[ str, str ] ] ]:
    """
    Aplica POS tagging a uma lista de textos (frases/versos).

    Args:
        text_list (list[str]): Lista de strings (frases ou versos).

    Returns:
        list[list[tuple[str, str]]]: Uma lista de sequências taggeadas.
                                     Cada sequência é uma lista de tuplas (palavra, tag).
    """
    tagged_sequences = [ ]
    # Processa os textos em lote para eficiência
    # A limpeza agressiva de pontuação deve vir DEPOIS ou ser feita pelo tagger
    docs = nlp.pipe( text_list )  # Processa a lista de textos

    for doc in docs:
        sequence = [ ]
        for token in doc:
            # token.text: a palavra/token original
            # token.pos_: a classe gramatical Universal Dependencies (mais simples, ex: NOUN, VERB, ADJ)
            # token.tag_: a classe gramatical mais detalhada, específica do corpus (ex: N|MAS|SG)
            # Vamos usar token.pos_ para simplificar
            if not token.is_punct and not token.is_space:  # Opcional: ignorar pontuação e espaços aqui
                sequence.append( (token.text.lower(), token.pos_) )  # Armazena palavra (minúscula) e tag
        if sequence:  # Adiciona apenas se a sequência não estiver vazia
            tagged_sequences.append( sequence )
    return tagged_sequences


In [11]:
train_ds = [ text.lower() for text in train_ds ]
test_ds = [ text.lower() for text in test_ds ]

tagged_train_sequences = tag_sequences( train_ds )
tagged_test_sequences = tag_sequences( test_ds )

## Construção do vocabulários, estados e mapeamento

In [12]:
import numpy as np

all_words = set()
all_tags = set()

for sequence in tagged_train_sequences:
    for word, tag in sequence:
        all_words.add( word )  # Adiciona palavra ao set
        all_tags.add( tag )  # Adiciona tag ao set

# Ordenar para ter índices consistentes
vocabulary = sorted( list( all_words ) )
states = sorted( list( all_tags ) )

# Criar mapeamentos para índices (essencial para matrizes numpy)
word_to_index = { word: i for i, word in enumerate( vocabulary ) }
tag_to_index = { tag: i for i, tag in enumerate( states ) }

n_states = len( states )
n_vocab = len( vocabulary )

print( f"Número de estados (tags) únicos: {n_states}" )
print( f"Tamanho do vocabulário: {n_vocab}" )

# print("Estados:", states)
# print("Vocabulário (primeiros 50):", vocabulary[:50])

Número de estados (tags) únicos: 16
Tamanho do vocabulário: 11162


In [13]:
# --- 2. Calcular Probabilidades Iniciais (π) ---

from collections import Counter

initial_tag_counts = Counter()
total_sequences = len( tagged_train_sequences )

for sequence in tagged_train_sequences:
    if sequence:  # Verifica se a sequência não está vazia
        first_tag = sequence[ 0 ][ 1 ]  # Pega a tag da primeira tupla (palavra, tag)
        initial_tag_counts[ first_tag ] += 1

# Inicializa o vetor pi com zeros (ou uma pequena probabilidade para suavização - veja nota)
pi = np.zeros( n_states )
for tag, count in initial_tag_counts.items():
    if tag in tag_to_index:  # Garante que a tag está no nosso conjunto de estados
        pi[ tag_to_index[ tag ] ] = count

# Normalizar para obter probabilidades
if total_sequences > 0:
    pi = pi / total_sequences
else:
    # Caso não haja sequências, distribui uniformemente (ou define como zero)
    pi = np.ones( n_states ) / n_states

print( f"Vetor Pi (Prob. Iniciais) calculado. Shape: {pi.shape}" )

Vetor Pi (Prob. Iniciais) calculado. Shape: (16,)


In [14]:
# --- 3. Calcular Matriz de Transição (A) ---

# Matriz A[i, j] = P(tag_j | tag_i)
# Usaremos contagens e depois normalizaremos

# Adicionar suavização de Laplace (add-alpha smoothing) é recomendado
# para evitar probabilidades zero para transições não vistas.
alpha_smooth = 0.1  # Um pequeno valor de suavização

# Contagem de transições (tag_i -> tag_j)
# Inicializa com alpha para suavização
transition_counts = np.full( (n_states, n_states), alpha_smooth )

# Contagem total de vezes que cada tag_i origina uma transição
# Inicializa com alpha * n_states para consistência na normalização com suavização
tag_origin_counts = np.full( n_states, alpha_smooth * n_states )

for sequence in tagged_train_sequences:
    for i in range( len( sequence ) - 1 ):  # Itera até a penúltima tupla
        current_tag = sequence[ i ][ 1 ]
        next_tag = sequence[ i + 1 ][ 1 ]

        if current_tag in tag_to_index and next_tag in tag_to_index:
            idx_current = tag_to_index[ current_tag ]
            idx_next = tag_to_index[ next_tag ]

            transition_counts[ idx_current, idx_next ] += 1
            tag_origin_counts[ idx_current ] += 1  # Incrementa a contagem de origem

# Normalizar para obter a matriz A de probabilidades
# A[i, j] = count(tag_i -> tag_j) / count(tag_i como origem)
A = transition_counts / tag_origin_counts[ :, np.newaxis ]  # Divide cada linha pelo total de origem correspondente

# Verifica se as linhas somam 1 (ou muito próximo devido a float precision)
# print("Soma das linhas da Matriz A (deve ser próximo de 1):", np.sum(A, axis=1))
print( f"Matriz A (Transição) calculada. Shape: {A.shape}" )

Matriz A (Transição) calculada. Shape: (16, 16)


In [15]:
# --- 4. Calcular Matriz de Emissão (B) ---

# Matriz B[j, k] = P(palavra_k | tag_j)
# Usaremos contagens e depois normalizaremos

# Contagem de emissões (tag_j -> palavra_k)
# Inicializa com alpha para suavização
emission_counts = np.full( (n_states, n_vocab), alpha_smooth )

# Contagem total de vezes que cada tag_j aparece
# Inicializa com alpha * n_vocab para consistência
tag_total_counts = np.full( n_states, alpha_smooth * n_vocab )

for sequence in tagged_train_sequences:
    for word, tag in sequence:
        if tag in tag_to_index and word in word_to_index:
            idx_tag = tag_to_index[ tag ]
            idx_word = word_to_index[ word ]

            emission_counts[ idx_tag, idx_word ] += 1
            tag_total_counts[ idx_tag ] += 1  # Incrementa a contagem total da tag

# Normalizar para obter a matriz B de probabilidades
# B[j, k] = count(tag_j emitindo palavra_k) / count(total de tag_j)
B = emission_counts / tag_total_counts[ :, np.newaxis ]  # Divide cada linha pelo total da tag correspondente

# Verifica se as linhas somam 1 (ou muito próximo)
# print("Soma das linhas da Matriz B (deve ser próximo de 1):", np.sum(B, axis=1))
print( f"Matriz B (Emissão) calculada. Shape: {B.shape}" )

# --- Resumo do que foi calculado ---
# vocabulary: list[str] - Lista única de palavras
# states: list[str] - Lista única de tags POS
# word_to_index: dict[str, int] - Mapeamento palavra -> índice
# tag_to_index: dict[str, int] - Mapeamento tag -> índice
# pi: np.ndarray (shape n_states) - Vetor de probabilidades iniciais
# A: np.ndarray (shape n_states x n_states) - Matriz de transição de estados
# B: np.ndarray (shape n_states x n_vocab) - Matriz de emissão

Matriz B (Emissão) calculada. Shape: (16, 11162)


## Mapeamentos Inversos

Para podermos interpretar os resultados (converter índices de volta para palavras e tags).

In [16]:
# Assumindo que você já tem word_to_index e tag_to_index

index_to_word = { index: word for word, index in word_to_index.items() }
index_to_tag = { index: tag for tag, index in tag_to_index.items() }

print( "Mapeamentos inversos criados." )

Mapeamentos inversos criados.


## Lógica de Predição

A ideia principal é calcular a probabilidade de cada palavra do vocabulário aparecer na posição mascarada, considerando o contexto (a tag da palavra anterior).



In [18]:
import numpy as np


# Importe o spacy e carregue o modelo nlp novamente se estiver em um novo script/sessão
# import spacy
# nlp = spacy.load("pt_core_news_lg") # Ou o modelo que você usou

def predict_masked_word( sentence_tokens: list[ str ],
                         mask_token: str = "MASK",
                         pi: np.ndarray = pi,
                         A: np.ndarray = A,
                         B: np.ndarray = B,
                         word_to_index: dict = word_to_index,
                         tag_to_index: dict = tag_to_index,
                         index_to_word: dict = index_to_word,
                         index_to_tag: dict = index_to_tag,
                         nlp = nlp
                         ):  # Passa o objeto nlp do spaCy
    """
    Prevê a palavra mais provável para substituir o token de máscara em uma sentença.

    Args:
        sentence_tokens (list[str]): Lista de palavras da sentença com um token de máscara.
                                     Ex: ['eu', 'amo', 'MASK']
        mask_token (str): O token que representa a máscara.
        pi (np.ndarray): Vetor de probabilidades iniciais.
        A (np.ndarray): Matriz de transição.
        B (np.ndarray): Matriz de emissão.
        word_to_index (dict): Mapeamento palavra -> índice.
        tag_to_index (dict): Mapeamento tag -> índice.
        index_to_word (dict): Mapeamento índice -> palavra.
        index_to_tag (dict): Mapeamento índice -> tag.
        nlp: Objeto spaCy carregado para fazer POS tagging.

    Returns:
        str: A palavra prevista mais provável.
        (Ou None se não puder prever)
    """
    try:
        mask_index = sentence_tokens.index( mask_token )
    except ValueError:
        print( f"Erro: Token de máscara '{mask_token}' não encontrado na sentença." )
        return None

    n_states = A.shape[ 0 ]
    n_vocab = B.shape[ 1 ]

    # --- Passo 1: Determinar as probabilidades da TAG na posição MASK ---

    # Probabilidade de cada estado (tag) na posição da máscara, P(tag_mask)
    state_probabilities_at_mask = np.zeros( n_states )

    if mask_index == 0:
        # Se a máscara é a primeira palavra, usamos as probabilidades iniciais (pi)
        state_probabilities_at_mask = pi
    else:
        # Se não for a primeira, usamos a matriz de transição (A)
        prev_word = sentence_tokens[ mask_index - 1 ]

        # Precisamos da tag da palavra anterior. Usamos o spaCy.
        # É melhor taggear a parte conhecida da sentença para dar contexto ao spaCy
        known_text = " ".join( sentence_tokens[ :mask_index ] )
        doc = nlp( known_text )

        if not doc or not doc[ -1 ].pos_ in tag_to_index:
            print(
                    f"Aviso: Não foi possível obter uma tag válida para a palavra anterior '{prev_word}'. Usando distribuição uniforme para tags."
            )
            # Se não conseguirmos a tag anterior, podemos assumir uma distribuição uniforme
            # ou usar alguma outra heurística. Uniforme é mais simples.
            state_probabilities_at_mask = np.ones( n_states ) / n_states
        else:
            prev_tag = doc[ -1 ].pos_
            prev_tag_idx = tag_to_index[ prev_tag ]
            # As probabilidades de transição da tag anterior para qualquer tag atual
            state_probabilities_at_mask = A[ prev_tag_idx, : ]

    # --- Passo 2: Calcular a probabilidade de cada PALAVRA na posição MASK ---

    # P(palavra | contexto) = Σ [ P(palavra | tag_mask) * P(tag_mask | contexto) ]
    # onde P(tag_mask | contexto) são as state_probabilities_at_mask que calculamos
    # e P(palavra | tag_mask) vem da matriz B

    # Inicializa as probabilidades das palavras
    word_probabilities = np.zeros( n_vocab )

    for current_tag_idx in range( n_states ):
        prob_current_tag = state_probabilities_at_mask[ current_tag_idx ]
        if prob_current_tag > 0:  # Otimização: só calcula se a tag tiver chance de ocorrer
            # Probabilidades de emissão para a tag atual P(palavra | tag_atual)
            emission_probs_for_tag = B[ current_tag_idx, : ]
            # Adiciona a contribuição ponderada pela probabilidade da tag
            word_probabilities += prob_current_tag * emission_probs_for_tag

    # --- Passo 3: Encontrar a palavra mais provável ---
    if np.sum( word_probabilities ) == 0:
        # Isso pode acontecer se todas as probabilidades intermediárias forem zero
        print( "Aviso: Não foi possível calcular probabilidades para as palavras." )
        return None  # Ou retornar uma palavra padrão

    # Encontra o índice da palavra com maior probabilidade
    # Evita prever o próprio token de máscara se ele estiver no vocabulário
    if mask_token in word_to_index:
        mask_vocab_idx = word_to_index[ mask_token ]
        word_probabilities[ mask_vocab_idx ] = 0  # Zera a prob. do token MASK

    # Poderia também zerar a probabilidade de tokens desconhecidos ou raros se necessário

    predicted_word_idx = np.argmax( word_probabilities )

    # Converte o índice de volta para a palavra
    predicted_word = index_to_word[ predicted_word_idx ]

    return predicted_word


In [24]:

frase: str = "Eu amo que MASK "
frase_exemplo = frase.split()
palavra_prevista = predict_masked_word( frase_exemplo )

if palavra_prevista:
    print( f"Frase: {' '.join( frase_exemplo )}" )
    print( f"Palavra Prevista: {palavra_prevista}" )

frase_exemplo_2 = [ "MASK", "é", "bom" ]
palavra_prevista_2 = predict_masked_word( frase_exemplo_2 )

if palavra_prevista_2:
    print( f"Frase: {' '.join( frase_exemplo_2 )}" )
    print( f"Palavra Prevista: {palavra_prevista_2}" )

Frase: Eu amo que MASK
Palavra Prevista: eu
Frase: MASK é bom
Palavra Prevista: não


In [25]:
import numpy as np


# Importe o spacy e carregue o modelo nlp novamente se estiver em um novo script/sessão
# import spacy
# nlp = spacy.load("pt_core_news_lg") # Ou o modelo que você usou

def predict_masked_word_with_tag_probs( sentence_tokens: list[ str ],
                                        mask_token: str = "MASK",
                                        top_n_tags: int = 5,  # Quantas tags mostrar
                                        pi: np.ndarray = pi,
                                        A: np.ndarray = A,
                                        B: np.ndarray = B,
                                        word_to_index: dict = word_to_index,
                                        tag_to_index: dict = tag_to_index,
                                        index_to_word: dict = index_to_word,
                                        index_to_tag: dict = index_to_tag,
                                        nlp = nlp
                                        ):
    """
    Prevê a palavra mais provável para substituir o token de máscara e
    exibe as tags POS mais prováveis para essa posição.

    Args:
        sentence_tokens (list[str]): Lista de palavras da sentença com um token de máscara.
        mask_token (str): O token que representa a máscara.
        top_n_tags (int): Número de tags mais prováveis a serem exibidas.
        pi, A, B, word_to_index, tag_to_index, index_to_word, index_to_tag: Parâmetros do HMM.
        nlp: Objeto spaCy carregado para fazer POS tagging.

    Returns:
        str: A palavra prevista mais provável.
        (Ou None se não puder prever)
    """
    try:
        mask_index = sentence_tokens.index( mask_token )
    except ValueError:
        print( f"Erro: Token de máscara '{mask_token}' não encontrado na sentença." )
        return None

    n_states = A.shape[ 0 ]
    n_vocab = B.shape[ 1 ]

    # --- Passo 1: Determinar as probabilidades da TAG na posição MASK ---
    state_probabilities_at_mask = np.zeros( n_states )
    prev_tag_info = "N/A (Primeira Palavra)"  # Info para display

    if mask_index == 0:
        state_probabilities_at_mask = pi
    else:
        prev_word = sentence_tokens[ mask_index - 1 ]
        known_text = " ".join( sentence_tokens[ :mask_index ] )
        doc = nlp( known_text )

        if not doc or not doc[ -1 ].pos_ in tag_to_index:
            print(
                f"Aviso: Não foi possível obter uma tag válida para a palavra anterior '{prev_word}'. Usando distribuição uniforme para tags."
                )
            state_probabilities_at_mask = np.ones( n_states ) / n_states
            prev_tag_info = f"{prev_word} (Tag Desconhecida)"
        else:
            prev_tag = doc[ -1 ].pos_
            prev_tag_idx = tag_to_index[ prev_tag ]
            state_probabilities_at_mask = A[ prev_tag_idx, : ]
            prev_tag_info = f"{prev_word} ({prev_tag})"  # Guarda info da tag anterior

    # --- Passo EXTRA: Exibir as Tags Mais Prováveis ---
    print( "-" * 30 )
    print( f"Análise para MASK na frase: {' '.join( sentence_tokens )}" )
    print( f"Palavra Anterior (Tag): {prev_tag_info}" )
    print( f"Top {top_n_tags} Tags Mais Prováveis para MASK:" )

    # Cria lista de (tag, probabilidade)
    tag_probs = [ ]
    for idx, prob in enumerate( state_probabilities_at_mask ):
        if idx in index_to_tag:  # Garante que o índice existe no mapeamento
            tag_name = index_to_tag[ idx ]
            tag_probs.append( (tag_name, prob) )

    # Ordena pela probabilidade (decrescente)
    sorted_tags = sorted( tag_probs, key = lambda item: item[ 1 ], reverse = True )

    # Exibe as top N
    for i in range( min( top_n_tags, len( sorted_tags ) ) ):
        tag, probability = sorted_tags[ i ]
        print( f"  {i + 1}. {tag:<10} Prob: {probability:.4f}" )  # :<10 alinha a tag à esquerda
    print( "-" * 30 )

    # --- Passo 2: Calcular a probabilidade de cada PALAVRA na posição MASK ---
    word_probabilities = np.zeros( n_vocab )
    for current_tag_idx in range( n_states ):
        prob_current_tag = state_probabilities_at_mask[ current_tag_idx ]
        if prob_current_tag > 0:
            emission_probs_for_tag = B[ current_tag_idx, : ]
            word_probabilities += prob_current_tag * emission_probs_for_tag

    # --- Passo 3: Encontrar a palavra mais provável ---
    if np.sum( word_probabilities ) == 0:
        print( "Aviso: Não foi possível calcular probabilidades para as palavras." )
        return None

    if mask_token in word_to_index:
        mask_vocab_idx = word_to_index[ mask_token ]
        word_probabilities[ mask_vocab_idx ] = 0

    predicted_word_idx = np.argmax( word_probabilities )
    predicted_word = index_to_word[ predicted_word_idx ]

    return predicted_word


# --- Exemplo de Uso ---
frase_exemplo = [ "o", "sol", "brilha", "e", "o", "céu", "está", "MASK" ]
palavra_prevista = predict_masked_word_with_tag_probs( frase_exemplo, top_n_tags = 5
                                                       )  # Mostra as 5 tags mais prováveis

if palavra_prevista:
    # A função agora imprime a tabela de tags internamente
    print( f"Palavra Prevista Final: {palavra_prevista}" )
    print( "\n" )

frase_exemplo_2 = [ "MASK", "sempre", "volta" ]
palavra_prevista_2 = predict_masked_word_with_tag_probs( frase_exemplo_2, top_n_tags = 5 )

if palavra_prevista_2:
    print( f"Palavra Prevista Final: {palavra_prevista_2}" )
    print( "\n" )

frase_exemplo_3 = [ "ela", "gosta", "de", "MASK", "chocolate" ]  # Máscara no meio
palavra_prevista_3 = predict_masked_word_with_tag_probs( frase_exemplo_3, top_n_tags = 5 )

if palavra_prevista_3:
    print( f"Palavra Prevista Final: {palavra_prevista_3}" )
    print( "\n" )

------------------------------
Análise para MASK na frase: o sol brilha e o céu está MASK
Palavra Anterior (Tag): está (AUX)
Top 5 Tags Mais Prováveis para MASK:
  1. VERB       Prob: 0.2917
  2. ADJ        Prob: 0.1330
  3. DET        Prob: 0.1153
  4. ADV        Prob: 0.1032
  5. PRON       Prob: 0.1014
------------------------------
Palavra Prevista Final: que


------------------------------
Análise para MASK na frase: MASK sempre volta
Palavra Anterior (Tag): N/A (Primeira Palavra)
Top 5 Tags Mais Prováveis para MASK:
  1. VERB       Prob: 0.1994
  2. PRON       Prob: 0.1759
  3. ADV        Prob: 0.1572
  4. DET        Prob: 0.1018
  5. NOUN       Prob: 0.0949
------------------------------
Palavra Prevista Final: não


------------------------------
Análise para MASK na frase: ela gosta de MASK chocolate
Palavra Anterior (Tag): de (ADP)
Top 5 Tags Mais Prováveis para MASK:
  1. NOUN       Prob: 0.5432
  2. DET        Prob: 0.1926
  3. PRON       Prob: 0.1450
  4. ADV        Prob: